In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import os
import gc

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import Ridge
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score

# model
import lightgbm as lgb

SEED=42
LABEL=None

In [0]:
path = 'drive/My Drive/bigcontest2019/'
os.chdir(path)

In [0]:
train_activity = pd.read_csv('data/train/train_activity.csv')
train_combat = pd.read_csv('data/train/train_combat.csv')
train_payment = pd.read_csv('data/train/train_payment.csv')
train_pledge = pd.read_csv('data/train/train_pledge.csv')
train_trade = pd.read_csv('data/train/train_trade.csv')

test1_activity = pd.read_csv('data/test/test1_activity.csv')
test1_combat = pd.read_csv('data/test/test1_combat.csv')
test1_payment = pd.read_csv('data/test/test1_payment.csv')
test1_pledge = pd.read_csv('data/test/test1_pledge.csv')
test1_trade = pd.read_csv('data/test/test1_trade.csv')

# test2_activity = pd.read_csv(path + 'test/test2_activity.csv')
# test2_combat = pd.read_csv(path + 'test/test2_combat.csv')
# test2_payment = pd.read_csv(path + 'test/test2_payment.csv')
# test2_pledge = pd.read_csv(path + 'test/test2_pledge.csv')
# test2_trade = pd.read_csv(path + 'test/test2_trade.csv')

train_label = pd.read_csv('data/train/train_label.csv')
train_label['total_spent'] = train_label['survival_time']*train_label['amount_spent']
train_label['adjust_survival_time'] = train_label['survival_time']
train_label.loc[train_label['adjust_survival_time']<64, 'adjust_survival_time'] = 1
train_label.loc[train_label['adjust_survival_time']==64, 'adjust_survival_time'] = 0
train_label = pd.merge(train_activity[['acc_id', 'char_id']], train_label, how='left', on='acc_id')

In [0]:
def feature_extraction(data, feature, method, on=ON):
    '''
    data - DataFrame
    feature - column name
    metod - ex) sum, unique, min, max ...etc
    '''
    
    standard_feature = 'custom_acc_id'
    if on==standard_feature:
        if method=='nunique':
            data = data.groupby(standard_feature)[feature].nunique().reset_index()
        elif method=='sum':
            data = data.groupby(standard_feature)[feature].sum().reset_index()
        elif method=='min':
            data = data.groupby(standard_feature)[feature].min().reset_index()
        elif method=='max':
            data = data.groupby(standard_feature)[feature].max().reset_index()
        elif method=='mean':
            data = data.groupby(standard_feature)[feature].mean().reset_index()
        elif method=='median':
            data = data.groupby(standard_feature)[feature].median().reset_index()
        elif method=='skew':
            data = data.groupby(standard_feature)[feature].skew().reset_index()
        elif method=='kurt':
            data = data.groupby(standard_feature)[feature].kurt().reset_index()
        elif method=='size':
            data = data.groupby(standard_feature)[feature].size().reset_index()
        else:
            return print('method is no return')
    else:
        standard_feature = 'acc_id'
        if method=='nunique':
            data = data.groupby(standard_feature)[feature].nunique().reset_index()
        elif method=='sum':
            data = data.groupby(standard_feature)[feature].sum().reset_index()
        elif method=='min':
            data = data.groupby(standard_feature)[feature].min().reset_index()
        elif method=='max':
            data = data.groupby(standard_feature)[feature].max().reset_index()
        elif method=='mean':
            data = data.groupby(standard_feature)[feature].mean().reset_index()
        elif method=='median':
            data = data.groupby(standard_feature)[feature].median().reset_index()
        elif method=='skew':
            data = data.groupby(standard_feature)[feature].skew().reset_index()
        elif method=='kurt':
            data = data.groupby(standard_feature)[feature].kurt().reset_index()
        elif method=='size':
            data = data.groupby(standard_feature)[feature].size().reset_index()
        else:
            return print('method is no return')
    
    return data

## two case
- train_payment > 0
- train_paymetn = 0

### train_payment > 0

In [0]:
train_label['custom_acc_id'] = train_label['acc_id'].astype(str)+'_'+train_label['char_id'].astype(str)
train_activity['custom_acc_id'] = train_activity['acc_id'].astype(str)+'_'+train_activity['char_id'].astype(str)
train_combat['custom_acc_id'] = train_combat['acc_id'].astype(str)+'_'+train_combat['char_id'].astype(str)
train_pledge['custom_acc_id'] = train_pledge['acc_id'].astype(str)+'_'+train_pledge['char_id'].astype(str)
train_trade['source_custom_acc_id'] = train_trade['source_acc_id'].astype(str)+'_'+train_trade['source_char_id'].astype(str)
train_trade['target_custom_acc_id'] = train_trade['target_acc_id'].astype(str)+'_'+train_trade['target_char_id'].astype(str)

In [0]:
train = pd.DataFrame(np.unique(train_label['custom_acc_id']), columns=['custom_acc_id'])

## Activity

In [0]:
DATA = train_activity
ON = 'custom_acc_id'
# train = pd.merge(train, feature_extraction(data=DATA, feature='char_id', method='nunique'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA, feature='server', method='nunique'), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='playtime', method='sum'), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='npc_kill', method='sum'), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='solo_exp', method='sum'), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='party_exp', method='sum'), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='quest_exp', method='sum'), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='rich_monster', method='sum'), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='death', method='sum'), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='revive', method='sum'), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='exp_recovery', method='sum'), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='fishing', method='sum'), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='private_shop', method='sum'), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='game_money_change', method='sum'), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='enchant_count', method='sum'), how='left', on=ON)

## Combat

In [0]:
DATA = train_combat
train = pd.merge(train, feature_extraction(data=DATA, feature='char_id', method='nunique'), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='server', method='nunique'), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='class', method='nunique'), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='level', method='nunique'), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='level', method='sum'), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='pledge_cnt', method='sum'), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='random_attacker_cnt', method='sum'), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='random_defender_cnt', method='sum'), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='temp_cnt', method='sum'), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='same_pledge_cnt', method='sum'), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='etc_cnt', method='sum'), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='num_opponent', method='sum'), how='left', on=ON)

## Payment

In [0]:
train['acc_id'] = train['custom_acc_id'].apply(lambda x: x.split('_')[0]).astype(int)
ON = 'acc_id'
DATA = train_payment
train = pd.merge(train, feature_extraction(data=DATA, feature='day', method='nunique', on=ON), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='day', method='size', on=ON), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='amount_spent', method='nunique', on=ON), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='amount_spent', method='sum', on=ON), how='left', on=ON)

## Pledge

In [0]:
DATA = train_pledge
ON = 'custom_acc_id'
train = pd.merge(train, feature_extraction(data=DATA, feature='char_id', method='nunique', on=ON), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='server', method='nunique', on=ON), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='pledge_id', method='nunique', on=ON), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='play_char_cnt', method='sum', on=ON), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='play_char_cnt', method='size', on=ON), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='combat_char_cnt', method='sum', on=ON), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='pledge_combat_cnt', method='sum', on=ON), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='random_attacker_cnt', method='sum', on=ON), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='random_defender_cnt', method='sum', on=ON), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='same_pledge_cnt', method='sum', on=ON), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='temp_cnt', method='sum', on=ON), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='etc_cnt', method='sum', on=ON), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='combat_play_time', method='sum', on=ON), how='left', on=ON)
train = pd.merge(train, feature_extraction(data=DATA, feature='non_combat_play_time', method='sum', on=ON), how='left', on=ON)

## Trade

In [0]:
DATA = train_trade
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'source_acc_id':'acc_id'}), feature='time', method='nunique'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'source_acc_id':'acc_id'}), feature='type', method='nunique'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'source_acc_id':'acc_id'}), feature='server', method='nunique'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'source_acc_id':'acc_id'}), feature='type', method='nunique'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'source_acc_id':'acc_id'}), feature='source_char_id', method='nunique'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'source_acc_id':'acc_id'}), feature='target_acc_id', method='nunique'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'source_acc_id':'acc_id'}), feature='target_char_id', method='nunique'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'source_acc_id':'acc_id'}), feature='item_type', method='nunique'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'source_acc_id':'acc_id'}), feature='item_amount', method='sum'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'source_acc_id':'acc_id'}), feature='item_amount', method='min'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'source_acc_id':'acc_id'}), feature='item_amount', method='max'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'source_acc_id':'acc_id'}), feature='item_amount', method='mean'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'source_acc_id':'acc_id'}), feature='item_price', method='sum'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'source_acc_id':'acc_id'}), feature='item_price', method='min'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'source_acc_id':'acc_id'}), feature='item_price', method='max'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'source_acc_id':'acc_id'}), feature='item_price', method='mean'), how='left', on='acc_id')

train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'target_acc_id':'acc_id'}), feature='time', method='nunique'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'target_acc_id':'acc_id'}), feature='type', method='nunique'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'target_acc_id':'acc_id'}), feature='server', method='nunique'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'target_acc_id':'acc_id'}), feature='type', method='nunique'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'target_acc_id':'acc_id'}), feature='source_acc_id', method='nunique'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'target_acc_id':'acc_id'}), feature='source_char_id', method='nunique'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'target_acc_id':'acc_id'}), feature='target_char_id', method='nunique'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'target_acc_id':'acc_id'}), feature='item_type', method='nunique'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'target_acc_id':'acc_id'}), feature='item_amount', method='sum'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'target_acc_id':'acc_id'}), feature='item_amount', method='min'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'target_acc_id':'acc_id'}), feature='item_amount', method='max'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'target_acc_id':'acc_id'}), feature='item_amount', method='mean'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'target_acc_id':'acc_id'}), feature='item_price', method='sum'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'target_acc_id':'acc_id'}), feature='item_price', method='min'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'target_acc_id':'acc_id'}), feature='item_price', method='max'), how='left', on='acc_id')
train = pd.merge(train, feature_extraction(data=DATA.rename(columns={'target_acc_id':'acc_id'}), feature='item_price', method='mean'), how='left', on='acc_id')

## Model

In [0]:
from sklearn.ensemble import RandomForestClassifier

def lgb_model(train, valid, train_label, test, seed=42, label=None):
    if label=='adjust_survival_time':
        trn_label = train_label.loc[train.index]
        val_label = train_label.loc[valid.index]
                
        rf = RandomForestClassifier().fit(train.drop(columns='custom_acc_id'), trn_label)
        preds = rf.predict(valid.drop(columns='custom_acc_id'))

    elif label=='survival_time':
        trn_label = train_label.loc[train.index]
        val_label = train_label.loc[valid.index]

        rf = RandomForestClassifier().fit(train.drop(columns='custom_acc_id'), trn_label)
        preds = rf.predict(test.drop(columns='custom_acc_id'))
    else:
        trn_label = train_label.loc[train.index]
        val_label = train_label.loc[valid.index]

        rf = RandomForestClassifier().fit(train.drop(columns='custom_acc_id'), trn_label)
        preds = rf.predict(test.drop(columns='custom_acc_id'))

    return preds

In [0]:
SEED=42
LABEL='survival_time'
data = pd.merge(train, train_label[[LABEL, 'custom_acc_id']].drop_duplicates(), how='left', on='custom_acc_id').fillna(-99)
# train_df, valid_df = train_test_split(train, test_size=0.2, random_state=SEED, shuffle=True, stratify=data[LABEL])
# preds = lgb_model(train_df, valid_df, data[LABEL], seed=SEED, label=LABEL)

In [0]:
LABEL='survival_time'
data = pd.merge(train, train_label[[LABEL, 'custom_acc_id']].drop_duplicates(), how='left', on='custom_acc_id').fillna(-99)
rf = RandomForestClassifier(n_estimators=100, random_state=42).fit(data.drop(columns=['custom_acc_id', LABEL]), data[LABEL])
preds = rf.predict(test.drop(columns=['custom_acc_id', LABEL]).fillna(-99))

In [0]:
LABEL='amount_spent'
data = pd.merge(train, train_label[[LABEL, 'custom_acc_id']].drop_duplicates(), how='left', on='custom_acc_id').fillna(-99)
rf2 = RandomForestRegressor(n_estimators=100, random_state=42n_jobs=-1).fit(data.drop(columns=['custom_acc_id', LABEL]), data[LABEL])
preds2 = rf2.predict(test.drop(columns=['custom_acc_id', 'survival_time']).fillna(-99))

In [0]:
test['survival_time'] = preds

In [0]:
test['amount_spent'] = preds2

In [0]:
aa = test.groupby('acc_id')['survival_time'].agg(lambda x: x.value_counts().index[0]).reset_index()

In [0]:
bb = test.groupby('acc_id')['amount_spent'].sum().reset_index()

In [0]:
ppap = pd.merge(aa, bb, how='left', on='acc_id')

In [0]:
ppap.to_csv('/content/pred.csv', index=False)

In [0]:
from scripts.metrics.score_function import score_function
# 예측, 실제

In [0]:
score_function(train_label[train_label['acc_id'].isin(valid_df_111.index)].groupby('acc_id').first().reset_index()[['acc_id', 'survival_time', 'amount_spent']], 
               train_label[train_label['acc_id'].isin(valid_df_111.index)].groupby('acc_id').first().reset_index()[['acc_id', 'survival_time', 'amount_spent']])

43214.51924831115


43214.51924831115

In [0]:
bb = train_label[train_label['acc_id'].isin(valid_df_111.index)].groupby('acc_id').first().reset_index()[['acc_id', 'survival_time', 'amount_spent']]

In [0]:
score_function(pd.merge(bb, valid_df_111.reset_index(), how='left', on='acc_id').drop(columns='survival_time').rename(columns={'preds':'survival_time'}), 
               train_label[train_label['acc_id'].isin(valid_df_111.index)].groupby('acc_id').first().reset_index()[['acc_id', 'survival_time', 'amount_spent']])

11853.189581627134


11853.189581627134

# predict

## Activity

In [0]:
test = pd.DataFrame(np.unique(test1_activity['acc_id'].astype(str)+'_'+test1_activity['char_id'].astype(str)), columns=['custom_acc_id'])

In [0]:
test1_activity['custom_acc_id'] = test1_activity['acc_id'].astype(str)+'_'+test1_activity['char_id'].astype(str)
test1_combat['custom_acc_id'] = test1_combat['acc_id'].astype(str)+'_'+test1_combat['char_id'].astype(str)
test1_pledge['custom_acc_id'] = test1_pledge['acc_id'].astype(str)+'_'+test1_pledge['char_id'].astype(str)
test1_trade['source_custom_acc_id'] = test1_trade['source_acc_id'].astype(str)+'_'+test1_trade['source_char_id'].astype(str)
test1_trade['target_custom_acc_id'] = test1_trade['target_acc_id'].astype(str)+'_'+test1_trade['target_char_id'].astype(str)

In [0]:
DATA = test1_activity
ON = 'custom_acc_id'
test = pd.merge(test, feature_extraction(data=DATA, feature='server', method='nunique'), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='playtime', method='sum'), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='npc_kill', method='sum'), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='solo_exp', method='sum'), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='party_exp', method='sum'), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='quest_exp', method='sum'), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='rich_monster', method='sum'), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='death', method='sum'), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='revive', method='sum'), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='exp_recovery', method='sum'), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='fishing', method='sum'), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='private_shop', method='sum'), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='game_money_change', method='sum'), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='enchant_count', method='sum'), how='left', on=ON)

## Combat

In [0]:
DATA = test1_combat
test = pd.merge(test, feature_extraction(data=DATA, feature='char_id', method='nunique'), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='server', method='nunique'), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='class', method='nunique'), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='level', method='nunique'), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='level', method='sum'), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='pledge_cnt', method='sum'), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='random_attacker_cnt', method='sum'), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='random_defender_cnt', method='sum'), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='temp_cnt', method='sum'), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='same_pledge_cnt', method='sum'), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='etc_cnt', method='sum'), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='num_opponent', method='sum'), how='left', on=ON)

## Payment

In [0]:
test['acc_id'] = test['custom_acc_id'].apply(lambda x: x.split('_')[0]).astype(int)
ON = 'acc_id'
DATA = test1_payment
test = pd.merge(test, feature_extraction(data=DATA, feature='day', method='nunique', on=ON), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='day', method='size', on=ON), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='amount_spent', method='nunique', on=ON), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='amount_spent', method='sum', on=ON), how='left', on=ON)

## Pledge

In [0]:
DATA = test1_pledge
ON = 'custom_acc_id'
test = pd.merge(test, feature_extraction(data=DATA, feature='char_id', method='nunique', on=ON), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='server', method='nunique', on=ON), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='pledge_id', method='nunique', on=ON), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='play_char_cnt', method='sum', on=ON), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='play_char_cnt', method='size', on=ON), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='combat_char_cnt', method='sum', on=ON), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='pledge_combat_cnt', method='sum', on=ON), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='random_attacker_cnt', method='sum', on=ON), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='random_defender_cnt', method='sum', on=ON), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='same_pledge_cnt', method='sum', on=ON), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='temp_cnt', method='sum', on=ON), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='etc_cnt', method='sum', on=ON), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='combat_play_time', method='sum', on=ON), how='left', on=ON)
test = pd.merge(test, feature_extraction(data=DATA, feature='non_combat_play_time', method='sum', on=ON), how='left', on=ON)

In [0]:
rf

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)